<a href="https://colab.research.google.com/github/allanzhou777/Adversarial-Attacks/blob/main/Transfer_Learning_on_Robust_Imagenet_on_Brain_Tumor_Detection_v1_0_Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install cox
!pip install tensorboardX
!pip install apex
!git clone https://github.com/MadryLab/robustness.git
!pip install dill
!pip install pyngrok

import os
os.chdir("/content/robustness/")
import torch
import torch as ch
from torch import nn

import dill

from cox.utils import Parameters
from cox.readers import CollectionReader
import cox.store
import copy
from robustness import imagenet_models # or cifar_models
from robustness import model_utils, train, defaults

from robustness.attacker import AttackerModel
from robustness.datasets import ImageNet
from robustness.datasets import DATASETS
# from robustness.datasets import MyNewDataSet
from robustness.model_utils import make_and_restore_model
from robustness.defaults import check_and_fill_args
from robustness.tools import constants, helpers
from robustness.tools.custom_modules import SequentialWithArgs
from torchvision.models import resnet18
from robustness.model_utils import make_and_restore_model
from argparse import ArgumentParser
import datetime
from fractions import Fraction
import plotly.graph_objects as go
import pandas as pd

CURRENT_DIRECTORY = os.getcwd()
print(f'current directory{CURRENT_DIRECTORY}')

# CONSTANTS
IMAGES_PATH = '/content/dataset/brain_tumor_dataset/'


# unzip Radiography dataset
!cp /content/drive/MyDrive/Adversarial\ Attacks/datasets/Brain\ Tumor\ Detection/Brain-Tumor-Detection.zip /content/
!unzip /content/Brain-Tumor-Detection.zip -d /content/dataset/

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 24.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

/content/robustness/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


current directory/content/robustness
Archive:  /content/Brain-Tumor-Detection.zip
  inflating: /content/dataset/brain_tumor_dataset/no/1 no.jpeg  
  inflating: /content/dataset/brain_tumor_dataset/no/10 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/11 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/12 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/13 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/14 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/15 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/17 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/18 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/19 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/2 no.jpeg  
  inflating: /content/dataset/brain_tumor_dataset/no/20 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/21 no.jpg  
  inflating: /content/dataset/brain_tumor_dataset/no/22 no.jpg  
  inflat

## Helper Functions


In [ ]:
#first bracket: eps-train; second bracket: eps-test
L2_EVAL = 'l2_eval_{}_{}/'
L_INF_EVAL = 'l_inf_eval_{}_{}/'
UNCONSTRAINED_EVAL = 'unconstrained_eval/'
FOURIER_EVAL = 'fourier_eval/'
RANDOM_EVAL = 'random_eval/'


#HELPER FUNCTIONS
calc_attack_lr = lambda eps_test, steps: 2.5 * eps_test / steps


def setup_store_with_metadata(args):
    '''
    Sets up a store for training according to the arguments object. See the
    argparse object above for options.
    '''
    # Create the store
    store = cox.store.Store(args.out_dir, args.exp_name)
    args_dict = args.__dict__
    schema = cox.store.schema_from_dict(args_dict)
    store.add_table('metadata', schema)
    store['metadata'].append_row(args_dict)

    return store


def evaluate_robust_model(eval_args,
                          model,
                          test_out_dir,
                          attack_steps=[20]):
  '''
  Evaluates a robust neural network on adversarial attacks. Stores results in Store in test_out_dir.
  Args:
    eval_args: (cox.utils.Parameters) hyperparameters used to evaluate the robust model
        https://github.com/MadryLab/robustness/blob/master/robustness/defaults.py#L91
    model: (robustness.attacker.AttackerModel) robust neural network
    test_out_dir: (str) where evaluation results are stored
    attack_steps: (optional, list) stores options for number of attack steps
  Returns: None
  '''
  assert eval_args.__contains__('eps'), "eps value required in eval_args parameter"
  assert eval_args.__contains__('constraint'), "constraint value required in eval_args parameter"

  for atk_steps in attack_steps:
    print('evaluating robust model with constaint: {}, eps: {}, and attack steps: {}'.format(eval_args.constraint,
                                                              Fraction(eval_args.eps).limit_denominator(),
                                                              atk_steps))

    eval_args.__setattr__('attack_steps', atk_steps)
    eval_args.__setattr__('attack_lr', calc_attack_lr(eval_args.__getattr__('eps'),
                                                    eval_args.__getattr__('attack_steps')))
    eval_args.__setattr__('out_dir', test_out_dir)

    # check whether the evaluation args provided are correct
    train.check_required_args(eval_args, eval_only=True)
    test_loader = copy.copy(val_loader)
    print(f'test results dir: {test_out_dir}')

    test_store = setup_store_with_metadata(eval_args)

    train.eval_model(eval_args, model, test_loader, test_store)
    test_store.close()

In [ ]:
def load_split_train_test(args, datadir, valid_size = .2):
    train_data = datasets.ImageFolder(datadir,
                    transform=data_transforms['train']) #Picks up Image Paths from its respective folders and label them
    test_data = datasets.ImageFolder(datadir,
                    transform=data_transforms['val'])
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    dataset_size = {"train":len(train_idx), "val":len(test_idx)}
    train_sampler = SubsetRandomSampler(train_idx) # Sampler for splitting train and val images
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = ch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=args.batch_size) # DataLoader provides data from traininng and validation in batches
    testloader = ch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=args.batch_size)
    return trainloader, testloader, dataset_size

In [ ]:
def ft(model_name, model_ft, num_classes, additional_hidden=0):
    if model_name in ["resnet", "resnet18", "resnet50", "wide_resnet50_2", "wide_resnet50_4", "resnext50_32x4d", 'shufflenet']:
        num_ftrs = model_ft.fc.in_features
        # The two cases are split just to allow loading
        # models trained prior to adding the additional_hidden argument
        # without errors
        if additional_hidden == 0:
            model_ft.fc = nn.Linear(num_ftrs, num_classes)
        else:
            model_ft.fc = SequentialWithArgs(
                *list(sum([[nn.Linear(num_ftrs, num_ftrs), nn.ReLU()] for i in range(additional_hidden)], [])),
                nn.Linear(num_ftrs, num_classes)
            )
        input_size = 224
    elif model_name == "alexnet":
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif "vgg" in model_name:
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif model_name == "squeezenet":
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "densenet":
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name in ["mnasnet", "mobilenet"]:
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        raise ValueError("Invalid model type, exiting...")

    return model_ft

In [ ]:
eval_args = Parameters({
    'adv_eval': 1,
    'eval_only': 1,
    'use_best': True,
    'random_restarts': 0,
})

In [ ]:
BLUE_SHADE = 'rgba(0, 0, 255, .2)'
RED_SHADE = 'rgba(255, 0, 0, .2)'
YELLOW_SHADE = 'rgba(255,255,0,0.2)'
PURPLE_SHADE = 'rgba(216,191,216,0.2)'
GREEN_SHADE = 'rgba(0,128,0, .2)'
ORANGE_SHADE = 'rgba(255,128,0, .2)'


colors = ['red', 'blue', 'green', 'orange']
def measurement_scatter(mean: pd.Series,
                        x: pd.Series,
                        upper: pd.Series,
                        lower: pd.Series,
                        col: str,
                        title: str,
                        color: str):


  shade = BLUE_SHADE
  if color == 'red':
    shade = RED_SHADE
  elif color == 'yellow':
    shade = YELLOW_SHADE
  elif color == 'purple':
    shade = PURPLE_SHADE
  elif color == 'green':
    shade = GREEN_SHADE
  elif color == 'orange':
    shade = ORANGE_SHADE
  return [go.Scatter(
        x=x,
        y=mean,
        line=dict(color=color),
        mode='lines+markers',
        name=title,
    )]

In [ ]:
#parser
parser = ArgumentParser()
parser.add_argument('-f')
parser = defaults.add_args_to_parser(defaults.MODEL_LOADER_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.TRAINING_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.PGD_ARGS, parser)
# Note that we can add whatever extra arguments we want to the parser here
args = parser.parse_args()

In [ ]:
#Statistics Based on ImageNet Data for Normalisation
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

data_transforms = {"train":transforms.Compose([
                                transforms.Resize((150,150)), #Resizes all images into same dimension
                                # transforms.RandomRotation(10), # Rotates the images upto Max of 10 Degrees
                                # transforms.RandomHorizontalFlip(p=0.4), #Performs Horizantal Flip over images
                                transforms.ToTensor(), # Coverts into Tensors
                                #transforms.Normalize(mean = mean_nums, std=std_nums)
                    ]), # Normalizes
                    "val": transforms.Compose([
                                transforms.Resize((150,150)),
                                transforms.CenterCrop(150), #Performs Crop at Center and resizes it to 150x150
                                transforms.ToTensor(),
                                # transforms.Normalize(mean=mean_nums, std = std_nums)
                    ])}

In [ ]:
ARCH = 'resnet_18'
OUT_DIR = '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/' + ARCH

# Train Model

In [ ]:
#define default training parameters
train_kwargs = {
    'adv_train': 0,
    'dataset': 'imagenet',
    'out_dir': OUT_DIR,
    'arch': 'resnet18',
    'epochs': 20,
    'mixed_precision': 0,
    'batch_size': 32,
    'log_iters': 1,
    'lr': 1e-3,
    'additional_hidden': 1,
}
args = Parameters(train_kwargs)
#sanity checks
ds_class = DATASETS[args.dataset]
assert args.dataset is not None, "Must provide a dataset"
args = check_and_fill_args(args, defaults.TRAINING_ARGS, ds_class)

if args.adv_train or args.adv_eval:
  args = check_and_fill_args(args, defaults.PGD_ARGS, ds_class)
args = check_and_fill_args(args, defaults.MODEL_LOADER_ARGS, ds_class)
print(f'Training Arguments: ', args)
print(train_kwargs)

Training Arguments:  {
  "adv_train": 0,
  "dataset": "imagenet",
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18",
  "arch": "resnet18",
  "epochs": 20,
  "mixed_precision": 0,
  "batch_size": 32,
  "log_iters": 1,
  "lr": 0.001,
  "additional_hidden": 1,
  "weight_decay": 0.0001,
  "momentum": 0.9,
  "step_lr": 50,
  "step_lr_gamma": 0.1,
  "lr_interpolation": "step",
  "save_ckpt_iters": -1,
  "data": "/tmp/",
  "workers": 30,
  "resume_optimizer": 0,
  "data_aug": 1
}
{'adv_train': 0, 'dataset': 'imagenet', 'out_dir': '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18', 'arch': 'resnet18', 'epochs': 20, 'mixed_precision': 0, 'batch_size': 32, 'log_iters': 1, 'lr': 0.001, 'additional_hidden': 1, 'weight_decay': 0.0001, 'momentum': 0.9, 'step_lr': 50, 'step_lr_gamma': 0.1, 'lr_interpolation': 'step', 'save_ckpt_iters': -1, 'data': '/tmp/', 'workers': 30, 'resume_optimizer': 0, 'data_aug': 1}


In [ ]:
train_data = datasets.ImageFolder(IMAGES_PATH,
                    transform=data_transforms['train']) #Picks up Image Paths from its respective folders and label them
test_data = datasets.ImageFolder(IMAGES_PATH,
                    transform=data_transforms['val'])

train_loader, val_loader, dataset_size = load_split_train_test(args, IMAGES_PATH, .2)
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}
class_names = train_loader.dataset.classes
print(class_names)

['no', 'yes']


# Define Model

##Training the model

In [ ]:
dataset = ImageNet(data_path = "")
# dataset = MyNewDataSet('/path/to/dataset/') #custom dataset path here
imagenet = dataset

# model_path = "resnet18_l2_eps0.ckpt"
# model_path = "resnet18_l2_eps0_5.ckpt"
# model_path = "resnet18_l2_eps1.ckpt"
model_path = "resnet18_l2_eps3.ckpt"
# model_path = "resnet18_l2_eps5.ckpt"

full_model_path = "/content/drive/MyDrive/Adversarial Attacks/Robust ImageNet Transfer Models/" + model_path
model, _ = make_and_restore_model(arch = resnet18(False), dataset = dataset, resume_path = full_model_path, add_custom_forward=False, pytorch_pretrained=False)
checkpoint = None

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning:

Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.



=> loading checkpoint '/content/drive/MyDrive/Adversarial Attacks/Robust ImageNet Transfer Models/resnet18_l2_eps3.ckpt'
=> loaded checkpoint '/content/drive/MyDrive/Adversarial Attacks/Robust ImageNet Transfer Models/resnet18_l2_eps3.ckpt' (epoch 90)


In [ ]:
# if not args.no_replace_last_layer and not args.eval_only:
print(f'[Replacing the last layer with {args.additional_hidden} '
      f'hidden layers and 1 classification layer that fits the {args.dataset} dataset.]')
while hasattr(model, 'model'):
    model = model.model
model = ft("resnet", model, 2, False)
model, checkpoint = model_utils.make_and_restore_model(arch=model, dataset=imagenet,
                                                        add_custom_forward=args.additional_hidden > 0 or False)

[Replacing the last layer with 1 hidden layers and 1 classification layer that fits the imagenet dataset.]


In [ ]:
try:
  st = datetime.datetime.now()

  out_store = cox.store.Store(OUT_DIR)

  train.train_model(args, model, (train_loader, val_loader), store=out_store)

  end = datetime.datetime.now()
  total = end - st
  print(total)
  out_store.close()
except Exception as e:
  out_store.close()
  print(f"model did not train to completion. \n error: {e}")

Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18/af9f2089-c4b7-4754-9412-241760d5ebf5


Train Epoch:0 | Loss 0.6663 | NatPrec1 59.606 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 7/7 [00:01<00:00,  3.57it/s]
Val Epoch:0 | Loss 0.5233 | NatPrec1 82.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  3.30it/s]
Train Epoch:1 | Loss 0.4050 | NatPrec1 83.251 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 7/7 [00:01<00:00,  4.12it/s]
Val Epoch:1 | Loss 0.3821 | NatPrec1 86.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  8.62it/s]
Train Epoch:2 | Loss 0.2674 | NatPrec1 89.163 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 7/7 [00:01<00:00,  4.60it/s]
Val Epoch:2 | Loss 0.3246 | NatPrec1 86.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  9.40it/s]
Train Epoch:3 | Loss 0.1977 | NatPrec1 92.611 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 7/7 [00:01<00:00,  4.85it/s]
Val Epoch:3 | Loss 0.3111 | NatPrec1 86.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|█

0:00:53.830737


# Experiment Information

EXP IDs


ResNet18 L2 eps= 0.0
808d86b6-86d7-40c6-a58e-de2a020151a3

ResNet18 L2 eps= 0.5
a8fb44ca-b61a-44bb-b6af-ab6492f4efb3

ResNet18 L2 eps= 1.0
8d4879b4-6733-489b-8aa4-d6e2ad1e083a

ResNet18 L2 eps= 3.0
af9f2089-c4b7-4754-9412-241760d5ebf5

ResNet18 L2 eps= 5.0




In [ ]:
EXP = "af9f2089-c4b7-4754-9412-241760d5ebf5"
BEST = "checkpoint.pt.best"
LATEST = "checkpoint.pt.latest"
FIRST = "0_checkpoint.pt"

In [ ]:
reader = CollectionReader(OUT_DIR)
logs = reader.df('logs')
reader.close()
logs = logs[logs.exp_id == EXP]
logs.head()

100%|██████████| 38/38 [00:00<00:00, 88.40it/s]


,epoch,nat_prec1,adv_prec1,nat_loss,adv_loss,train_prec1,train_loss,time,exp_id
0,1,82.0,-1.0,0.523324,-1.0,59.605911,0.666254,2.617564,af9f2089-c4b7-4754-9412-241760d5ebf5
1,2,86.0,-1.0,0.382055,-1.0,83.251228,0.404987,5.870825,af9f2089-c4b7-4754-9412-241760d5ebf5
2,3,86.0,-1.0,0.324613,-1.0,89.162560,0.267388,9.285315,af9f2089-c4b7-4754-9412-241760d5ebf5
3,4,86.0,-1.0,0.311120,-1.0,92.610832,0.197719,11.265604,af9f2089-c4b7-4754-9412-241760d5ebf5
4,5,86.0,-1.0,0.293921,-1.0,96.059113,0.153131,15.314352,af9f2089-c4b7-4754-9412-241760d5ebf5


In [ ]:
# validation accuracies and loss
nat_prec1 = logs.nat_prec1
adv_prec1 = logs.adv_prec1
nat_loss = logs.nat_loss
adv_loss = logs.adv_loss

# training accuracies and losses --> here, we only have adversarial accuracies and losses...do you understand why?
train_prec1 = logs.train_prec1
train_loss = logs.train_loss

In [ ]:
losses = ['train_loss', 'nat_loss', 'adv_loss']

scatter = []
for i, loss in enumerate(losses):
  scatter = scatter + measurement_scatter(logs[loss], logs.epoch, logs[loss], logs[loss], loss, loss, color=colors[i])


fig = go.Figure(scatter)

fig.update_layout(
    yaxis_title='CE Loss',
    xaxis_title='Epoch',
    hovermode="x",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font={'size': 20}
)

fig.show()

In [ ]:
accuracies = ['train_prec1', 'nat_prec1', 'adv_prec1']

scatter = []
for i, acc in enumerate(accuracies):
  print(f'Accuracy: {loss}...')
  scatter = scatter + measurement_scatter(logs[acc], logs.epoch, logs[acc], logs[acc], acc, acc, color=colors[i])


fig = go.Figure(scatter)

fig.update_layout(
    yaxis_title='Accuracy',
    xaxis_title='Epoch',
    hovermode="x",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font={'size': 20}
)

fig.show()

Accuracy: adv_loss...
Accuracy: adv_loss...
Accuracy: adv_loss...


#Evaluate Model

In [ ]:
resume_path=OUT_DIR + "/" + EXP + "/" + BEST

In [ ]:
from robustness import imagenet_models # or cifar_models
from robustness.datasets import ImageNet

model = AttackerModel(resnet18(), imagenet)

while hasattr(model, 'model'):
    model = model.model
model = ft("resnet", model, 2, False)

model, checkpoint = model_utils.make_and_restore_model(arch=model, dataset=imagenet,
                                                        add_custom_forward=args.additional_hidden > 0 or False,
                                                       resume_path = resume_path)

=> loading checkpoint '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18/af9f2089-c4b7-4754-9412-241760d5ebf5/checkpoint.pt.best'
=> loaded checkpoint '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18/af9f2089-c4b7-4754-9412-241760d5ebf5/checkpoint.pt.best' (epoch 9)


##Evaluating robust model with constaint: 2, ϵ-test: 0, 0.25, 0.5, 1.0, 2.0, and attack_steps: [20, 200]


####ϵ-test: 0


In [ ]:
# from pyngrok import ngrok

# # Set the ngrok authtoken (only needed for free users)
# NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open a tunnel to the TensorBoard port
# public_url = ngrok.connect(addr="6006", proto="http")

# public_url

In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 0.0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 0) + 'robustness'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 0.0,
  "attack_steps": 20,
  "attack_lr": 0.0125,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18fourier_eval/"
}
evaluating robust model with constaint: 2, eps: 0, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness/dc84af46-9285-4025-998c-ac594838953b


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  9.43it/s]
Val Epoch:0 | Loss 0.3114 | AdvPrec1 88.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:01<00:00,  1.08it/s]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness/5c987dcd-bdcb-4666-9874-fd2a30f49584 serve

####ϵ-test: 0.25


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 0.25)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 0.25) + 'robustness'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 0.25,
  "attack_steps": 20,
  "attack_lr": 0.0,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness"
}
evaluating robust model with constaint: 2, eps: 1/4, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness/e8e4752c-bde2-44dc-b2fb-4f4e8a6b8857


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00, 10.01it/s]
Val Epoch:0 | Loss 0.6399 | AdvPrec1 76.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness/7d3e5fe7-55e2-4bfe-a73c-514a55fee786 serve

####ϵ-test: 0.5


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 0.5)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 0.5) + 'take_one'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 0.5,
  "attack_steps": 20,
  "attack_lr": 0.03125,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness"
}
evaluating robust model with constaint: 2, eps: 1/2, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one/42c03c88-4ca6-4b06-b744-61cfbb4ebd07


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  7.14it/s]
Val Epoch:0 | Loss 1.1660 | AdvPrec1 58.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one/7292cdd1-37b7-40f6-b357-4ed01bb05c6c serve

####ϵ-test: 1.0


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 1.0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 1.0) + 'test_1.0'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 1.0,
  "attack_steps": 20,
  "attack_lr": 0.0625,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one"
}
evaluating robust model with constaint: 2, eps: 1, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0/92661294-d643-43b5-b83e-db642fada8fe


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  4.55it/s]
Val Epoch:0 | Loss 2.7679 | AdvPrec1 22.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0/d025d1f7-da12-4911-95f8-c58fb3eaa7e4 serve

####ϵ-test: 2.0


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 2.0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 2.0) + 'eps_2.0'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 2.0,
  "attack_steps": 20,
  "attack_lr": 0.125,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0"
}
evaluating robust model with constaint: 2, eps: 2, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0/08ac2667-e0cb-4ae1-b2c8-c37cc2a03952


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  7.13it/s]
Val Epoch:0 | Loss 5.5404 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:02<00:00,  1.10s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0/dc6118a3-cbea-4d48-a059-1cb4412730b8 serve

##Evaluating robust model with constaint: ∞, ϵ-test: 0, 8/255, 16/255, and attack_steps: [20, 200]


In [ ]:
# from pyngrok import ngrok

# # Set the ngrok authtoken (only needed for free users)
# NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open a tunnel to the TensorBoard port
# public_url = ngrok.connect(addr="6006", proto="http")

# public_url

####ϵ-test: 0


In [ ]:
eval_args.__setattr__('constraint', 'inf')
eval_args.__setattr__('eps', 0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_L_INF_OUT = OUT_DIR + L_INF_EVAL.format(8/255, 0) + 'linf'
evaluate_robust_model(eval_args, model, TEST_L_INF_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "inf",
  "eps": 0,
  "attack_steps": 20,
  "attack_lr": 0.25,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0"
}
evaluating robust model with constaint: inf, eps: 0, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf/7e3ebca9-3a44-4003-910d-85899e5de337


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00, 10.02it/s]
Val Epoch:0 | Loss 0.3114 | AdvPrec1 88.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:01<00:00,  1.09it/s]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf/11f4508d-1e4e-40fc-bc17-4a990637ef36 serve

####ϵ-test: 8/255


In [ ]:
eval_args.__setattr__('constraint', 'inf')
eval_args.__setattr__('eps', 8/255)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_L_INF_OUT = OUT_DIR + L_INF_EVAL.format(8/255, 8/255) + 'linf'
evaluate_robust_model(eval_args, model, TEST_L_INF_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "inf",
  "eps": 0.03137254901960784,
  "attack_steps": 20,
  "attack_lr": 0.0,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf"
}
evaluating robust model with constaint: inf, eps: 8/255, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf/b53eec79-1859-419f-b021-450e6c251e43


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00, 10.33it/s]
Val Epoch:0 | Loss 7.7243 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf/20696687-fdf7-43f1-82e8-0fd0151b0d58 serve

####ϵ-test: 16/255


In [ ]:
eval_args.__setattr__('constraint', 'inf')
eval_args.__setattr__('eps', 16/255)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_L_INF_OUT = OUT_DIR + L_INF_EVAL.format(8/255, 16/255) +'linf'
evaluate_robust_model(eval_args, model, TEST_L_INF_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "inf",
  "eps": 0.06274509803921569,
  "attack_steps": 20,
  "attack_lr": 0.00392156862745098,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf"
}
evaluating robust model with constaint: inf, eps: 16/255, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf/608e2209-0b7d-4b48-a181-ba813e9a7bd6


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00, 10.12it/s]
Val Epoch:0 | Loss 9.7705 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf/30aa6236-19a0-4830-b9bb-fba0d691d7c1 serve

##Evaluating robust model with constrain: unconstrained, ϵ-test: unconstrained, and attack_steps: [20, 100]


In [ ]:
from pyngrok import ngrok

# Set the ngrok authtoken (only needed for free users)
NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a tunnel to the TensorBoard port
public_url = ngrok.connect(addr="6006", proto="http")

public_url

<NgrokTunnel: "https://b137-35-240-181-35.ngrok-free.app" -> "http://localhost:6006">

In [ ]:
eval_args.__setattr__('constraint', 'unconstrained')
eval_args.__setattr__('eps', 0.1) #eps-values not used
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_UNCONSTRAINED_OUT = OUT_DIR + UNCONSTRAINED_EVAL + 'unconstrainedyeaa'
evaluate_robust_model(eval_args, model, TEST_UNCONSTRAINED_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "unconstrained",
  "eps": 0.1,
  "attack_steps": 20,
  "attack_lr": 0.00784313725490196,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf"
}
evaluating robust model with constaint: unconstrained, eps: 1/10, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa/f81255e9-c82d-44a6-94e3-5f14da48f583


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  6.85it/s]
Val Epoch:0 | Loss 0.3765 | AdvPrec1 88.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa/1458520a-efec-4c73-96f8-0ce62cbea338 serve

##Evaluating robust model with constrain: random smooth, ϵ-test: random smooth and attack_steps: [20, 100]


In [ ]:
# from pyngrok import ngrok

# # Set the ngrok authtoken (only needed for free users)
# NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open a tunnel to the TensorBoard port
# public_url = ngrok.connect(addr="6006", proto="http")

# public_url

In [ ]:
eval_args.__setattr__('constraint', 'random_smooth')
eval_args.__setattr__('eps', 0.1) #eps value not used
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_RANDOM_OUT = OUT_DIR + RANDOM_EVAL + 'randomsmoothyeaaa'
evaluate_robust_model(eval_args, model, TEST_RANDOM_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "random_smooth",
  "eps": 0.1,
  "attack_steps": 20,
  "attack_lr": 0.0125,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa"
}
evaluating robust model with constaint: random_smooth, eps: 1/10, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa/65fc89f4-bdd6-4dd9-995c-30677f6bda03


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  6.51it/s]
Val Epoch:0 | Loss 0.6614 | AdvPrec1 56.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:01<00:00,  1.57it/s]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa/90e42c70-baa0-40dc-8030-657abea311ac serve

##Evaluating robust model with constrain: fourier, ϵ-test: fourier and attack_steps: [20, 100]


In [ ]:
eval_args.__setattr__('constraint', 'fourier')
eval_args.__setattr__('eps', 0.1)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_FOURIER_OUT = OUT_DIR + FOURIER_EVAL
evaluate_robust_model(eval_args, model, TEST_FOURIER_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "fourier",
  "eps": 0.1,
  "attack_steps": 20,
  "attack_lr": 0.0125,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa"
}
evaluating robust model with constaint: fourier, eps: 1/10, and attack steps: 20
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18fourier_eval/
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18fourier_eval/ef325e89-df14-4956-a978-ea5c8dddaec6


Val Epoch:0 | Loss 0.3114 | NatPrec1 88.000 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 2/2 [00:00<00:00,  9.80it/s]
  0%|          | 0/2 [00:00<?, ?it/s]


AttributeError: ignored